In [6]:
cursor.close()
mydb.close()

In [1]:
import glob
import mysql.connector

from mysql.connector import (connection)

#now we create the table in the database we want to add

mydb = connection.MySQLConnection(user='root', password='password',
                                 host='127.0.0.1', database='VELA_GEN6')

#print(mydb)

cursor = mydb.cursor(buffered='True')



In [2]:
#this finds the Massive Central at the last timestep and makes all of the 

cursor.execute("SELECT DISTINCT VELA_number, scale_factor, rockstar_id, New_Rvir_Calculation from propermvircalc where scale_factor = (select max(scale_factor) from propermvircalc as p where p.VELA_number = propermvircalc.VELA_number) and (select count(*) from propermvircalc as f where f.scale_factor = propermvircalc.scale_factor and f.VELA_number = propermvircalc.VELA_number and f.New_Rvir_Calculation >= propermvircalc.New_Rvir_Calculation ) <= 1;")
myresult = cursor.fetchall()

ids = []

for x in myresult:
    ids.append((x[0], x[2]))
    print(x)
    

massive_centrals_id_list = []

for central_info in ids:
    VELA_number = central_info[0]
    starting_id = central_info[1]
    print(VELA_number)
    
    cursor.execute("SELECT rockstar_id, scale_factor, Last_mainleaf_depthfirst_ID from rockstar where VELA_number = {} and rockstar_id = {}".format(VELA_number, starting_id))
    depthfirst_id_search = cursor.fetchall()
    
    depthfirst_id = None
    for item in depthfirst_id_search:
        depthfirst_id = item[2]
    
    cursor.execute("SELECT VELA_number, scale_factor, rockstar_id from rockstar where VELA_number = {} and Last_mainleaf_depthfirst_ID = {}".format(VELA_number, depthfirst_id))
    massive_centrals_search = cursor.fetchall()
    
    for massive_central in massive_centrals_search:
        current_VELA_number = massive_central[0]
        current_scale_factor = massive_central[1]
        current_rockstar_id = massive_central[2]
        
        cursor.execute("UPDATE properSFRmerge SET projenitor_number=1 WHERE VELA_number={} AND scale_factor={} AND rockstar_id={}".format(current_VELA_number, current_scale_factor, current_rockstar_id))
        mydb.commit()
    
    
#now we find the ones not in the Massive Central and Find Those

import numpy as np

cursor.execute("SELECT VELA_number from properSFRmerge")
VELA_numbers_sql = cursor.fetchall()

VELA_numbers = np.unique(VELA_numbers_sql)
print(VELA_numbers)

for VELA_number in VELA_numbers:
    print(VELA_number)
    cursor.execute("SELECT VELA_number, scale_factor, rockstar_id from properSFRmerge WHERE VELA_number={} and projenitor_number IS NULL".format(VELA_number))
    myresult = cursor.fetchall()

    depthfirst_id_list = []
    rockstar_id_list = []

    for item in myresult:
        rockstar_id_list.append(item[2])
        
    #print(rockstar_id_list)

    cursor.execute("SELECT rockstar_id, scale_factor, Last_mainleaf_depthfirst_ID, VELA_number from rockstar where VELA_number = {} and rockstar_id IN {}".format(VELA_number, tuple(rockstar_id_list)))
    depthfirst_id_search = cursor.fetchall()

    for item1 in depthfirst_id_search:
        #print(item1)
        depthfirst_id_list.append(item1[2])

    print(depthfirst_id_list)
    
    ids, counts = np.unique(depthfirst_id_list, return_counts=True)
    print(depthfirst_id_list)
    print(ids)
    print(counts)

    sorted_counts = []

    for id1, counts1 in zip(ids,counts):
        sorted_counts.append([id1, counts1])

    sorted_counts = sorted(sorted_counts, key=lambda x: float(x[1]), reverse=False)

    print(sorted_counts)

    length = len(sorted_counts)

    count = 1

    while count <= length:
        current_depthfirst_id = sorted_counts[length-count][0]
        current_number_halos = sorted_counts[length-count][1]
        print(current_depthfirst_id, current_number_halos)

        if current_number_halos > 1:

            for halo in depthfirst_id_search:

                halo_depthfirst_id = halo[2]
                halo_scale_factor = halo[1]
                halo_rockstar_id = halo[0]

                if halo_depthfirst_id == current_depthfirst_id:

                    #this needs to be count + 1 since count of 1 is already taken
                    projenitor_number = count + 1

                    cursor.execute("UPDATE properSFRmerge SET projenitor_number={} WHERE VELA_number={} AND scale_factor={} AND rockstar_id={}".format(projenitor_number, VELA_number, halo_scale_factor, halo_rockstar_id))
                    mydb.commit()      

        count = count + 1
     

('01', 0.51, 255697, 124.57359680632553)
('02', 0.5, 187706, 113.17988406871555)
('03', 0.5, 161550, 108.15564372028615)
('04', 0.5, 114341, 87.87868389353079)
('05', 0.5, 109088, 83.47516694687116)
('06', 0.56, 1023572, 226.38525829970357)
('07', 0.56, 994966, 207.4389740993421)
('08', 0.56, 806006, 199.85450297985045)
('09', 0.56, 718322, 209.2784207695866)
('10', 0.56, 692195, 182.29704249465073)
('11', 0.56, 460604, 157.4912523865816)
('12', 0.56, 468095, 159.1637406279523)
('13', 0.56, 433074, 170.91912744595567)
('14', 0.56, 370852, 146.01404007029873)
('15', 0.56, 257180, 124.42367349758311)
('16', 0.31, 702099, 90.52056632363556)
('17', 0.34, 651491, 124.95573419910886)
('18', 0.33, 625748, 85.08359432017673)
('19', 0.33, 615020, 105.64013536788407)
('20', 0.5, 509211, 168.43280799661414)
('21', 0.5, 433461, 156.8649645830635)
('22', 0.5, 262232, 150.17578832086195)
('23', 0.5, 306200, 122.28483367111004)
('24', 0.5, 318415, 117.17357505727034)
('25', 0.51, 266663, 109.77238543

In [3]:
cursor.execute('CREATE TABLE MassiveCentrals_new (VELA_number CHAR(2), scale_factor float(53), rockstar_id INT(255), projenitor_number INT(255)) ')





In [5]:
import numpy as np

cursor.execute("SELECT VELA_number from properSFRmerge")
VELA_numbers_sql = cursor.fetchall()

VELA_numbers = np.unique(VELA_numbers_sql)
print(VELA_numbers)

for VELA_number in VELA_numbers:
    print(VELA_number)
    cursor.execute("SELECT VELA_number, scale_factor, rockstar_id from properSFRmerge WHERE VELA_number={} and projenitor_number IS NULL".format(VELA_number))
    myresult = cursor.fetchall()

    depthfirst_id_list = []
    rockstar_id_list = []

    for item in myresult:
        rockstar_id_list.append(item[2])
        
    #print(rockstar_id_list)

    cursor.execute("SELECT rockstar_id, scale_factor, Last_mainleaf_depthfirst_ID, VELA_number from rockstar where VELA_number = {} and rockstar_id IN {}".format(VELA_number, tuple(rockstar_id_list)))
    depthfirst_id_search = cursor.fetchall()

    for item1 in depthfirst_id_search:
        #print(item1)
        depthfirst_id_list.append(item1[2])

    print(depthfirst_id_list)
    
    ids, counts = np.unique(depthfirst_id_list, return_counts=True)
    print(depthfirst_id_list)
    print(ids)
    print(counts)

    sorted_counts = []

    for id1, counts1 in zip(ids,counts):
        sorted_counts.append([id1, counts1])

    sorted_counts = sorted(sorted_counts, key=lambda x: float(x[1]), reverse=False)

    print(sorted_counts)

    length = len(sorted_counts)

    count = 1

    while count <= length:
        current_depthfirst_id = sorted_counts[length-count][0]
        current_number_halos = sorted_counts[length-count][1]
        print(current_depthfirst_id, current_number_halos)

        if current_number_halos > 1:

            for halo in depthfirst_id_search:

                halo_depthfirst_id = halo[2]
                halo_scale_factor = halo[1]
                halo_rockstar_id = halo[0]

                if halo_depthfirst_id == current_depthfirst_id:

                    #this needs to be count + 1 since count of 1 is already taken
                    projenitor_number = count + 1

                    cursor.execute("UPDATE properSFRmerge SET projenitor_number={} WHERE VELA_number={} AND scale_factor={} AND rockstar_id={}".format(projenitor_number, VELA_number, halo_scale_factor, halo_rockstar_id))
                    mydb.commit()      

        count = count + 1


['01' '02' '03' '04' '05' '06' '07' '08' '09' '10' '11' '12' '13' '14'
 '15' '16' '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28'
 '29' '30' '31' '32' '33' '34' '35']
01
[23493, 56440, 23493, 56440, 23493, 56440, 23493, 56440, 56440, 23493, 56440, 23493, 56440, 23493, 56440, 23493, 56440, 23493, 56440, 23493, 56440, 64068, 23493, 56440, 64068, 23493, 56440, 23493, 56440, 23493, 56440, 23493, 56440, 23493]
[23493, 56440, 23493, 56440, 23493, 56440, 23493, 56440, 56440, 23493, 56440, 23493, 56440, 23493, 56440, 23493, 56440, 23493, 56440, 23493, 56440, 64068, 23493, 56440, 64068, 23493, 56440, 23493, 56440, 23493, 56440, 23493, 56440, 23493]
[23493 56440 64068]
[16 16  2]
[[64068, 2], [23493, 16], [56440, 16]]
56440 16
23493 16
64068 2
02
[25188, 25188, 59267, 15302, 25188, 59267, 15302, 25188, 59267, 15302, 25188, 59267, 25188, 59267, 25188, 59267, 25188, 32791, 59267, 25188, 32791, 59267, 65314, 25188, 32791, 59267, 65314, 25188, 32791, 59267, 65314, 25188, 32791, 59267, 25

KeyboardInterrupt: 

In [7]:
import numpy as np

ids, counts = np.unique(depthfirst_id_list, return_counts=True)
print(depthfirst_id_list)
print(ids)
print(counts)

sorted_counts = []

for id1, counts1 in zip(ids,counts):
    sorted_counts.append([id1, counts1])
    
sorted_counts = sorted(sorted_counts, key=lambda x: float(x[1]), reverse=True)

print(sorted_counts)

length = len(sorted_counts)

count = 1

while count <= length:
    current_depthfirst_id = sorted_counts[length-count][0]
    current_number_halos = sorted_counts[length-count][1]
    print(current_depthfirst_id, current_number_halos)
    
    if current_number_halos > 1:
        
        for halo in depthfirst_id_search:
            
            halo_depthfirst_id = halo[2]
            halo_scale_factor = halo[1]
            halo_rockstar_id = halo[0]
            
            if halo_depthfirst_id == current_depthfirst_id:
                
                #this needs to be count + 1 since count of 1 is already taken
                projenitor_number = count + 1
                
                cursor.execute("UPDATE properSFRmerge SET projenitor_number={} WHERE VELA_number={} AND scale_factor={} AND rockstar_id={}".format(projenitor_number, VELA_number, halo_scale_factor, halo_rockstar_id))
                mydb.commit()      
        
    count = count + 1





[25188, 25188, 59267, 15302, 25188, 59267, 15302, 25188, 59267, 15302, 25188, 59267, 25188, 59267, 25188, 59267, 25188, 32791, 59267, 25188, 32791, 59267, 65314, 25188, 32791, 59267, 65314, 25188, 32791, 59267, 65314, 25188, 32791, 59267, 25188, 32791, 59267, 25188, 32791, 59267, 25188, 59267, 59267, 59267]
[15302 25188 32791 59267 65314]
[ 3 15  7 16  3]
[[59267, 16], [25188, 15], [32791, 7], [15302, 3], [65314, 3]]


IndexError: list index out of range

In [12]:
massive_centrals_id_list = []

for central_info in ids:
    VELA_number = central_info[0]
    starting_id = central_info[1]
    print(VELA_number)
    
    cursor.execute("SELECT rockstar_id, scale_factor, Last_mainleaf_depthfirst_ID from rockstar where VELA_number = {} and rockstar_id = {}".format(VELA_number, starting_id))
    depthfirst_id_search = cursor.fetchall()
    
    depthfirst_id = None
    for item in depthfirst_id_search:
        depthfirst_id = item[2]
    
    cursor.execute("SELECT VELA_number, rockstar_id, scale_factor from rockstar where VELA_number = {} and Last_mainleaf_depthfirst_ID = {}".format(VELA_number, depthfirst_id))
    massive_centrals_search = cursor.fetchall()
    
    for item in massive_centrals_search:
        massive_centrals_id_list.append(item)
        print(item)
    
    
    
    
    break  
    

06
('06', 16672, 0.1)
('06', 34880, 0.102)
('06', 67993, 0.105)
('06', 101990, 0.107)
('06', 136663, 0.11)
('06', 172466, 0.112)
('06', 208991, 0.114)
('06', 246110, 0.117)
('06', 283591, 0.12)
('06', 321540, 0.122)
('06', 359868, 0.124)
('06', 398580, 0.127)
('06', 437549, 0.13)
('06', 476729, 0.132)
('06', 515779, 0.134)
('06', 555158, 0.137)
('06', 594590, 0.14)
('06', 634262, 0.142)
('06', 673807, 0.145)
('06', 713567, 0.147)
('06', 752705, 0.149)
('06', 792436, 0.15)
('06', 832094, 0.152)
('06', 871702, 0.155)
('06', 911106, 0.157)
('06', 950397, 0.16)
('06', 989337, 0.162)
('06', 1027869, 0.165)
('06', 1066046, 0.167)
('06', 1103962, 0.17)
('06', 1141288, 0.172)
('06', 1178398, 0.175)
('06', 1215304, 0.177)
('06', 1251511, 0.179)
('06', 1287984, 0.18)
('06', 1324366, 0.182)
('06', 1360369, 0.184)
('06', 1396039, 0.187)
('06', 1431639, 0.19)
('06', 1467124, 0.192)
('06', 1502175, 0.195)
('06', 1537012, 0.197)
('06', 1571583, 0.2)
('06', 1605836, 0.202)
('06', 1639674, 0.204)
('06'

In [39]:
tuple1 = []
for item in massive_centrals_id_list:
    tuple1.append(item[1])


cursor.execute("SELECT * from propermvircalc where VELA_number = 06 and rockstar_id in {}".format(tuple(tuple1)))   
final = cursor.fetchall()

for item in final:
    print(item)


('06', 0.127, 398580, 15214285973.383463, 10.481954597161575, 11.1001129235731, 0.0010694780733757625, 265964584.18216684, 1561729754.9909673, 15921107821.990427, 126530522.9184881, 280054560.4595193, 180027176.28599915, 180027176.28599915, 207357401.25506353, 471903369.6394712, 0.6577290151264531, 0.0010694780733764286, 0.8502350683341957, 0.0010694780733763176, 0.9464880949380667, 0.0010694780733764286)
('06', 0.13, 437549, 16185714561.355364, 10.91302010864905, 11.574415266748991, 0.0011134598621218572, 329088274.8116288, 1644596296.4117603, 16985298076.507261, 177185433.33707765, 206814549.85380974, 239327570.90403697, 239327570.90403697, 266043950.8993397, 389156956.92653346, 0.5756587487166166, 0.00111345986212108, 0.7627200055529639, 0.001113459862121191, 0.8584775536953798, 0.001113459862121191)
('06', 0.132, 476729, 17128571720.26927, 11.334627657015455, 12.040078414160579, 0.0011564766510563373, 355549106.07602966, 1718071961.009045, 18087242677.789013, 161572462.9130273, 892

In [35]:
cursor.close()
mydb.close()

In [ ]:
#this finds the largest rvir of each snapshot for each VELA, found here https://www.xaprb.com/blog/2006/12/07/how-to-select-the-firstleastmax-row-per-group-in-sql/

select rockstar_id, scale_factor, VELA_number, New_Rvir_Calculation
from propermvircalc
where (
   select count(*) from propermvircalc as f
   where f.scale_factor = propermvircalc.scale_factor and f.VELA_number = propermvircalc.VELA_number and f.New_Rvir_Calculation >= propermvircalc.New_Rvir_Calculation
) <= 1;